In [2]:
!pip install tensorflow==2.8.0
import tensorflow as tf
print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 668.3 MB 17 kB/s 
     |████████████████████████████████| 5.8 MB 26.4 MB/s 
     |████████████████████████████████| 462 kB 52.3 MB/s 
     |████████████████████████████████| 1.4 MB 67.4 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
2.8.0


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

Dataset link:
https://docs.google.com/spreadsheets/d/1qART38B2l_G7VkJKsjvkIFkDRekeOnTj/edit?usp=sharing&ouid=114253294372712711027&rtpof=true&sd=true

Linked to trained models: https://drive.google.com/drive/folders/1vOaNOT7qQYzozfA0Cce67sqyZ-8w9MlH?usp=sharing

In [4]:
! gdown --id 1qART38B2l_G7VkJKsjvkIFkDRekeOnTj

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1qART38B2l_G7VkJKsjvkIFkDRekeOnTj
To: /content/Voltage_BOI.xlsx
100% 282k/282k [00:00<00:00, 93.7MB/s]


In [5]:
df = pd.read_excel('Voltage_BOI.xlsx')
df=df.fillna(0)
df.head()

,B0('9-16'),C0('11-25'),A0('11-28'),B('11-11'),C0('9-20'),B0('9-27'),A0('10-03'),B('10-21'),C('10-24'),B('10-28'),...,B0('10-03'),C0('9-16'),C0('11-28'),C0('9-10'),C('10-30'),C(11-06'),C('11-11'),B0(9-24'),min,d
0,0.0159,0.0063,0.0061,0.0021,0.0039,0.0280,0.0103,0.0016,0.0009,0.0017,...,0.0116,0.0131,0.0046,0.0024,0.0075,0.0013,0.0013,0.0128,0.0,0.00
1,0.0582,0.0504,0.0021,0.0009,0.0052,0.0448,0.0014,0.0009,0.0008,0.0006,...,0.0022,0.0534,0.0027,0.0089,0.0046,0.0040,0.0011,0.0625,5.0,0.00
2,0.1085,0.0667,0.0036,0.0014,0.0191,0.0740,0.0010,0.0013,0.0014,0.0008,...,0.0016,0.0804,0.0060,0.0242,0.0025,0.0094,0.0027,0.1146,10.0,0.01
3,0.1339,0.0679,0.0075,0.0021,0.0500,0.0858,0.0009,0.0016,0.0024,0.0013,...,0.0017,0.1123,0.0098,0.0480,0.0064,0.0176,0.0050,0.1324,15.0,0.01
4,0.1448,0.0666,0.0133,0.0036,0.0789,0.0919,0.0010,0.0020,0.0039,0.0017,...,0.0019,0.1243,0.0154,0.0631,0.0126,0.0279,0.0078,0.1383,20.0,0.01


Definition of input and output data

In [9]:
def test_input_out_data(k):
  return np.array(df.iloc[0:k, 40:56]).transpose()

In [10]:
x_test_24 = test_input_out_data(289)

x_test_16 = test_input_out_data(193)

x_test_12 = test_input_out_data(145)

x_test_8 = test_input_out_data(97)

x_test_6 = test_input_out_data(73)

x_test_2 = test_input_out_data(25)

y_test = np.array(df.iloc[1511, 40:56]).transpose().astype('float64')

Loading Saved Models

In [11]:
def model(json_file, weights):
  with open(json_file, 'r') as f:
    loaded_model = tf.keras.models.model_from_json(f.read())
    loaded_model.load_weights(weights)
  return loaded_model

In [12]:
model24 = model('model24_direct.json', 'weights24_direct.h5')
model16 = model('model16_direct.json', 'weights16_direct.h5')
model12 = model('model12_direct.json', 'weights12_direct.h5')
model8 = model('model8_direct.json', 'weights8_direct.h5')
model6 = model('model6_direct.json', 'weights6_direct.h5')
model2 = model('model2_direct.json', 'weights2_direct.h5')

Prediction on a test sample

In [13]:
pred24 = np.array(model24(x_test_24))
pred16 = np.array(model16(x_test_16))
pred12 = np.array(model12(x_test_12))
pred8 = np.array(model8(x_test_8))
pred6 = np.array(model6(x_test_6))
pred2 = np.array(model2(x_test_2))

In [14]:
# error predict
def print_error(y, pred):
  print(mean_absolute_error(y, pred))
  print(mean_absolute_percentage_error(y, pred))
  TEST_MAE = []
  for i in range(0, 16):
    TEST_MAE.append(abs(y[i]-pred[i]))
  print(np.array(TEST_MAE).max())
  print(r2_score(y, pred))
  print(' ')

In [15]:
# error predict
print_error(y_test, pred24)
print_error(y_test, pred16)
print_error(y_test, pred12)
print_error(y_test, pred8)
print_error(y_test, pred6)
print_error(y_test, pred2)

1.4620272314548493
0.08724113294355122
9.617565
0.9230416343719431
 
1.975670636892319
0.11232977587994677
11.339924
0.8779836448730596
 
2.2432126021385197
0.13597497884253804
10.624981
0.8755757921239472
 
2.456458494663239
0.15769900609021265
9.469721
0.8574009094332378
 
3.5447854173183444
0.21020569403509592
18.741627
0.6594116641840735
 
4.6978434109687806
0.3660402170716154
25.36213
0.3855531456989024
 


BOD > 220 pollution

from liters to 60, 50, 40 ml

In [18]:
BOD = [(220 * 40)/ 1000, (220 * 60)/ 1000, (220 * 60)/ 1000, \
       (220 * 60)/ 1000, (220 * 60)/ 1000, (220 * 60)/ 1000, \
       (220 * 60)/ 1000, (220 * 60)/ 1000, (220 * 60)/ 1000, \
       (220 * 60)/ 1000, (220 * 60)/ 1000, (220 * 40)/ 1000, \
       (220 * 60)/ 1000, (220 * 60)/ 1000, (220 * 60)/ 1000, (220 * 50)/ 1000]

In [55]:
BOD

[8.8,
 13.2,
 13.2,
 13.2,
 13.2,
 13.2,
 13.2,
 13.2,
 13.2,
 13.2,
 13.2,
 8.8,
 13.2,
 13.2,
 13.2,
 11.0]

In [66]:
def pollution(y_test):
  pollution = []
  pred_pollution = []
  for i in range(0, 16):
    if y_test[i] > BOD[i]:
      pollution.append(1)
    else:
      pollution.append(0)
  return np.array(pollution)

In [74]:
pol = pollution(y_test)
pred_pol24 = pollution(pred24)
pred_pol16= pollution(pred16)
pred_pol12 = pollution(pred12)
pred_pol8 = pollution(pred8)
pred_pol6 = pollution(pred6)
pred_pol2 = pollution(pred2)

In [87]:
def count(pred_pol):
  c = 0
  for i in range(0, 16):
    if pred_pol[i] == pol[i]:
      c = c+1
  return c

In [81]:
def count_1(pred_pol):
  c = 0
  for i in range(0, 16):
    if pol[i] == 1:
      if pred_pol[i] == pol[i]:
        c = c+1
  return c

In [83]:
def count_0(pred_pol):
  c = 0
  for i in range(0, 16):
    if pol[i] == 0:
      if pred_pol[i] == pol[i]:
        c = c+1
  return c

sensitivity

In [82]:
print(count_1(pred_pol24) / pol[pol == 1].size)
print(count_1(pred_pol16) / pol[pol == 1].size)
print(count_1(pred_pol12) / pol[pol == 1].size)
print(count_1(pred_pol8) / pol[pol == 1].size)
print(count_1(pred_pol6) / pol[pol == 1].size)
print(count_1(pred_pol2) / pol[pol == 1].size)

1.0
0.9
0.9
0.8
0.8
0.8


specificity

In [84]:
print(count_0(pred_pol24) / pol[pol == 0].size)
print(count_0(pred_pol16) / pol[pol == 0].size)
print(count_0(pred_pol12) / pol[pol == 0].size)
print(count_0(pred_pol8) / pol[pol == 0].size)
print(count_0(pred_pol6) / pol[pol == 0].size)
print(count_0(pred_pol2) / pol[pol == 0].size)

1.0
0.8333333333333334
1.0
1.0
0.6666666666666666
0.6666666666666666


accuracy

In [88]:
print(count(pred_pol24) / pol.size)
print(count(pred_pol16) / pol.size)
print(count(pred_pol12) / pol.size)
print(count(pred_pol8) / pol.size)
print(count(pred_pol6) / pol.size)
print(count(pred_pol2) / pol.size)

1.0
0.875
0.9375
0.875
0.75
0.75
